In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import nrrd
import json
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache
from allensdk.api.queries.ontologies_api import OntologiesApi
import requests
from anatomy.anatomy_api import AnatomyApi

In [2]:
aapi = AnatomyApi()
ss = aapi.get_summary_structure_data('id')
mcc = MouseConnectivityCache(manifest_file = '/connectivity/mouse_connectivity_manifest.json')
structure_tree = mcc.get_structure_tree()
ia_map = structure_tree.get_id_acronym_map()
ai_map = {value:key for key, value in ia_map.items()}

ss_dict = dict(zip(ss, [ai_map[struct] for struct in ss]))

In [3]:
iso = structure_tree.get_structures_by_acronym(['Isocortex'])[0]
iso = structure_tree.descendant_ids([iso['id']])[0]
iso = [structure for structure in iso if structure in ss]
hipp = structure_tree.get_structures_by_acronym(['HPF'])[0]
hipp = structure_tree.descendant_ids([hipp['id']])[0]
hipp = [structure for structure in hipp if structure in ss]
olf = structure_tree.get_structures_by_acronym(['OLF'])[0]
olf = structure_tree.descendant_ids([olf['id']])[0]
olf = [structure for structure in olf if structure in ss]
th = structure_tree.get_structures_by_acronym(['TH'])[0]
th = structure_tree.descendant_ids([th['id']])[0]
th = [structure for structure in th if structure in ss]
mb = structure_tree.get_structures_by_acronym(['MB'])[0]
mb = structure_tree.descendant_ids([mb['id']])[0]
mb = [structure for structure in mb if structure in ss]
ctxsp = structure_tree.get_structures_by_acronym(['CTXsp'])[0]
ctxsp = structure_tree.descendant_ids([ctxsp['id']])[0]
ctxsp = [structure for structure in ctxsp if structure in ss]
STR = structure_tree.get_structures_by_acronym(['STR'])[0]
STR = structure_tree.descendant_ids([STR['id']])[0]
STR = [structure for structure in STR if structure in ss]
cb = structure_tree.get_structures_by_acronym(['CB'])[0]
cb = structure_tree.descendant_ids([cb['id']])[0]
cb = [structure for structure in cb if structure in ss]
p = structure_tree.get_structures_by_acronym(['P'])[0]
p = structure_tree.descendant_ids([p['id']])[0]
p = [structure for structure in p if structure in ss]
hy = structure_tree.get_structures_by_acronym(['HY'])[0]
hy = structure_tree.descendant_ids([hy['id']])[0]
hy = [structure for structure in hy if structure in ss]
my = structure_tree.get_structures_by_acronym(['MY'])[0]
my = structure_tree.descendant_ids([my['id']])[0]
my = [structure for structure in my if structure in ss]
pal = structure_tree.get_structures_by_acronym(['PAL'])[0]
pal = structure_tree.descendant_ids([pal['id']])[0]
pal = [structure for structure in pal if structure in ss]

In [4]:
major_divisions = []
major_divisions_full = []
for struct in ss_dict.keys():
    if struct in iso:
        major_divisions.append('Isocortex')
        major_divisions_full.append('Isocortex')
    elif struct in hipp:
        major_divisions.append('HPF')
        major_divisions_full.append('Hippocampal formation')
    elif struct in olf:
        major_divisions.append('OLF')
        major_divisions_full.append('Olfactory areas')
    elif struct in ctxsp:
        major_divisions.append('CTXsp')
        major_divisions_full.append('Cortical subplate')
    elif struct in th:
        major_divisions.append('TH')
        major_divisions_full.append('Thalamus')
    elif struct in STR:
        major_divisions.append('STR')
        major_divisions_full.append('Striatum')
    elif struct in pal:
        major_divisions.append('PAL')
        major_divisions_full.append('Pallidum')
    elif struct in hy:
        major_divisions.append('HY')
        major_divisions_full.append('Hypothalamus')
    elif struct in mb:
        major_divisions.append('MB')
        major_divisions_full.append('Midbrain')
    elif struct in p:
        major_divisions.append('P')
        major_divisions_full.append('Pons')
    elif struct in my:
        major_divisions.append('MY')
        major_divisions_full.append('Medulla')
    elif struct in cb:
        major_divisions.append('CB')
        major_divisions_full.append('Cerebellum')
    elif struct == 1009:
        major_divisions.append('fiber tracts')
        major_divisions_full.append('fiber tracts')
    else:
        print(struct)
        print('no major division found')

In [5]:
ss_df = pd.DataFrame(ss_dict, index = ['structure_abbreviation'])
ss_df = ss_df.T
ss_df.reset_index(inplace = True)
ss_df.rename(columns = {'index': 'structure_id'}, inplace = True)
ss_df['major_division'] = major_divisions_full
ss_df['major_division_abbreviation'] = major_divisions

In [6]:
ss_df.tail()

,structure_id,structure_abbreviation,major_division,major_division_abbreviation
311,989,FN,Cerebellum,CB
312,91,IP,Cerebellum,CB
313,846,DN,Cerebellum,CB
314,589508455,VeCB,Cerebellum,CB
315,1009,fiber tracts,fiber tracts,fiber tracts


In [8]:
ss_df.index = [structure_tree.get_structures_by_id([stid])[0]['graph_order'] for \
              stid in ss_df['structure_id'].values]
ss_df.sort_index(inplace = True)
#ss_df.to_csv('ss_major_division_mapping.csv')